### 1. CREAR BASE DE DATOS

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía.

In [1]:
import pandas as pd
import mysql.connector as con
import random
import datetime
from datetime import timedelta

Base de datos Supermercado

In [2]:
# Crear base de datos "supermercado"
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin"
)
cursor = conexion.cursor()
cursor.execute("DROP DATABASE IF EXISTS supermercado;")
cursor.execute("CREATE SCHEMA supermercado DEFAULT CHARACTER SET utf8;")
#cursor.close()
#connection.close()
#connection.close()

In [3]:
# Creo una sola tabla
# Conexion
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

# crear tabla
query_tabla_tiendas = """
CREATE TABLE IF NOT EXISTS tiendas (
    id_tienda INT AUTO_INCREMENT PRIMARY KEY,
    nombre_tienda VARCHAR(30),
    direccion_tienda VARCHAR(50),
    ciudad VARCHAR (50)
)
"""
# Hacer los cambios en la base de datos
cursor.execute(query_tabla_tiendas)
conexion.commit()
print("Tabla creada exitosamente.")

# Cerrar la conexion
cursor.close()
conexion.close()

Tabla creada exitosamente.


In [4]:
# Creo varias tablas a la vez
conexion = con.connect(
    host="localhost",
    port= "3306",
    user= "root",
    password= "admin",
    database= "supermercado"
)
cursor = conexion.cursor()

query_varias_tablas = ["""
    CREATE TABLE IF NOT EXISTS empleados (
    id_empleado INT AUTO_INCREMENT PRIMARY KEY,
    nombre_empleado VARCHAR(30),
    puesto VARCHAR(20),
    id_tienda INT,
    FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INT AUTO_INCREMENT PRIMARY KEY,
    nombre_categoria VARCHAR(40)
)""",

    """
    CREATE TABLE IF NOT EXISTS productos (
    id_producto INT AUTO_INCREMENT PRIMARY KEY,
    nombre_producto VARCHAR(40),
    precio DECIMAL (6, 2),
    stock INT,
    id_categoria INT,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria) ON DELETE CASCADE
)""",

    """
    CREATE TABLE IF NOT EXISTS clientes (
    id_cliente INT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(15),
    apellido VARCHAR (20),
    email VARCHAR (60),
    telefono INT,
    direccion VARCHAR (40),
    codigo_postal INT
)""",

    """
    CREATE TABLE IF NOT EXISTS ordenes (
    id_orden INT AUTO_INCREMENT PRIMARY KEY,
    id_cliente INT,
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente),
    id_empleado INT,
    FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado),
    fecha_orden DATE,
    metodo_pago ENUM('Tarjeta', 'Efectivo')
)""",

   """
    CREATE TABLE IF NOT EXISTS detalle_orden (
    id_detalle INT AUTO_INCREMENT PRIMARY KEY,
    id_orden INT NOT NULL,
    FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden) ON DELETE CASCADE,
    id_producto INT NOT NULL,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE CASCADE,
    cantidad INT (15) NOT NULL,
    descuento DECIMAL (3,2),
    precio_unitario DECIMAL(6,2) NOT NULL
)"""
]
# Para ejecutar cada consulta a kla base de datos y crear todas las tablas
for query in query_varias_tablas:
    cursor.execute(query)

conexion.commit()
print("Tablas creadas exitosamente.")

# Para copiar el "precio" de la tabla "productos" en la tabla "detalle_orden"
query_trigger = """
CREATE TRIGGER insertar_precio_unitario
BEFORE INSERT ON detalle_orden
FOR EACH ROW
BEGIN
    DECLARE precio_producto DECIMAL(6,2);

    -- Obtener el precio del producto desde la tabla productos
    SELECT precio INTO precio_producto 
    FROM productos 
    WHERE id_producto = NEW.id_producto;

    -- Asignar el precio al nuevo registro en detalle_orden
    SET NEW.precio_unitario = precio_producto;
END;
"""

cursor.execute(query_trigger)

conexion.commit()
print("Trigger creado exitosamente.")

# Para cerrar la conexión
cursor.close()
conexion.close()

Tablas creadas exitosamente.
Trigger creado exitosamente.


### 2. Generar datos demo desde Python

Tiendas

In [5]:
# Nombres de tiendas
filas = random.randint(11, 11)

nombre_tienda = [f'Tienda_0{numero}' if numero < 10 else f'Tienda_{numero}' for numero in range(filas)]
nombre_tienda

['Tienda_00',
 'Tienda_01',
 'Tienda_02',
 'Tienda_03',
 'Tienda_04',
 'Tienda_05',
 'Tienda_06',
 'Tienda_07',
 'Tienda_08',
 'Tienda_09',
 'Tienda_10']

In [6]:
# Direcciones 
direccion_tienda = [f'Calle_falsa_{numero}' if numero < 10 else f'Calle_falsa_{numero}' for numero in range(filas)]
direccion_tienda

['Calle_falsa_0',
 'Calle_falsa_1',
 'Calle_falsa_2',
 'Calle_falsa_3',
 'Calle_falsa_4',
 'Calle_falsa_5',
 'Calle_falsa_6',
 'Calle_falsa_7',
 'Calle_falsa_8',
 'Calle_falsa_9',
 'Calle_falsa_10']

In [7]:
# Lista de ciudades
ciudades = ["Madrid", "Barcelona", "Sevilla", "Valencia", "Bilbao", "Zaragoza", "Granada", "Alicante","Murcia","Albacete"]

# Número de ciudades aleatorias
num_ciudades = 10  

# Generar la lista de ciudades aleatorias
ciudades_aleatorias = [random.choice(ciudades) for _ in range(num_ciudades)]
print(ciudades_aleatorias)

['Madrid', 'Alicante', 'Albacete', 'Bilbao', 'Granada', 'Bilbao', 'Albacete', 'Barcelona', 'Granada', 'Valencia']


In [8]:
# Insertamos los datos generados anteriormente
# Conectar a MySQL
conexion2 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion2.cursor()

# Insertamos los datos en la tabla
for i in range(10):
    cursor.execute("INSERT INTO tiendas (nombre_tienda, direccion_tienda, ciudad) VALUES (%s, %s, %s)",
                   (nombre_tienda[i], direccion_tienda[i], ciudades_aleatorias[i]))

conexion2.commit()
cursor.close()
conexion2.close()
print("Las tiendas han sido insertadas correctamente")

Las tiendas han sido insertadas correctamente


Empleados

In [9]:
# Lista de puestos
puestos = ["Director tienda", "Panadero", "Frutero", "Charcutero", "Pescatero", "Cajero", "Reponedor", "Repartidor"]

# Número de puestos aleatorias
num_puestos = 200  

# Generar la lista de puestos aleatorios
puestos_aleatorios = [random.choice(puestos) for _ in range(num_puestos)]
print(puestos_aleatorios)

['Director tienda', 'Panadero', 'Frutero', 'Frutero', 'Pescatero', 'Pescatero', 'Frutero', 'Reponedor', 'Cajero', 'Charcutero', 'Cajero', 'Repartidor', 'Cajero', 'Charcutero', 'Director tienda', 'Reponedor', 'Frutero', 'Pescatero', 'Reponedor', 'Pescatero', 'Reponedor', 'Pescatero', 'Pescatero', 'Reponedor', 'Reponedor', 'Frutero', 'Pescatero', 'Panadero', 'Pescatero', 'Pescatero', 'Reponedor', 'Repartidor', 'Frutero', 'Reponedor', 'Cajero', 'Panadero', 'Reponedor', 'Reponedor', 'Frutero', 'Pescatero', 'Pescatero', 'Reponedor', 'Charcutero', 'Pescatero', 'Director tienda', 'Director tienda', 'Director tienda', 'Pescatero', 'Charcutero', 'Pescatero', 'Panadero', 'Pescatero', 'Frutero', 'Director tienda', 'Charcutero', 'Pescatero', 'Reponedor', 'Reponedor', 'Reponedor', 'Reponedor', 'Director tienda', 'Charcutero', 'Panadero', 'Reponedor', 'Reponedor', 'Cajero', 'Frutero', 'Panadero', 'Repartidor', 'Reponedor', 'Frutero', 'Charcutero', 'Charcutero', 'Pescatero', 'Repartidor', 'Panadero',

In [10]:
# Llenamos la tabla empleados
conexion4 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion4.cursor()

#Nombre empleados
nom_empleados = [f"Empleado{i}" for i in range(1, 201)]

# Lista de IDs de tiendas disponibles en la tabla "tiendas"
cursor.execute("SELECT id_tienda FROM tiendas")
id_tienda_disponibles = [row[0] for row in cursor.fetchall()]

# Inserto empleado con un puesto y un id_tienda aleatorio
for i in range(200):
    puesto_aleatorio = puestos_aleatorios[i]
    id_tienda_aleatorio = random.choice(id_tienda_disponibles)

    cursor.execute("INSERT INTO empleados (nombre_empleado, puesto, id_tienda) VALUES (%s, %s, %s)",
                   (nom_empleados[i],puesto_aleatorio, id_tienda_aleatorio))

conexion4.commit()
cursor.close()
conexion4.close()

print("Los 200 empleados han sido insertados correctamente con un puesto y una tienda aleatoria.")

Los 200 empleados han sido insertados correctamente con un puesto y una tienda aleatoria.


Categorias

In [11]:
# Insertar datos en la tabla categorias
import mysql.connector as con

# Conectar a MySQL
conexion5 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion5.cursor()

# Lista de categorías
categorias = ["Carnes", "Lacteos", "Frutas", "Panaderia", "Bebidas", 
              "Cereales", "Congelados", "Conservas", "Limpieza", "Belleza"]

# Insertar las categorías en la tabla
for categoria in categorias:
    cursor.execute("INSERT INTO categorias (nombre_categoria) VALUES (%s)", (categoria,))

# Confirmar cambios y cerrar conexión
conexion5.commit()
cursor.close()
conexion5.close()

print("Las 10 categorias se han insertado correctamente")

Las 10 categorias se han insertado correctamente


Productos

In [12]:
# Conectar a MySQL
conexion6 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion6.cursor()

# Diccionario de categorías y sus productos
productos_por_categoria = {
    "Lacteos": ["Leche", "Yogur griego natural", "Queso mozzarella", "Mantequilla"],
    "Carnes": ["Pechuga de pollo", "Carne picada", "Jamón", "Salchichas"],
    "Frutas": ["Manzana roja", "Plátano", "Zanahoria", "Lechuga"],
    "Panaderia": ["Pan integral", "Croissants", "Galletas", "Pastel de chocolate"],
    "Bebidas": ["Coca-Cola", "Zumo de naranja", "Café", "Agua mineral"],
    "Cereales": ["Arroz", "Lentejas", "Avena", "Fideos"],
    "Congelados": ["Helado de fresa", "Nuggets de pollo", "Espinacas congeladas", "Pizza congelada"],
    "Conservas": ["Atún", "Salsa de tomate", "Caballa", "Anchoas"],
    "Limpieza": ["Detergente líquido", "Jabón de manos", "Papel higiénico", "Desinfectante"],
    "Belleza": ["Champu", "Pasta de dientes", "Desodorante", "Crema hidratante"]
}

# IDs de las categorías existentes
cursor.execute("SELECT id_categoria, nombre_categoria FROM categorias")
categorias_disponibles = {nombre: id for id, nombre in cursor.fetchall()}

# Inserto productos con su categoría correspondiente
for categoria, lista_productos in productos_por_categoria.items():
    if categoria in categorias_disponibles:
        id_categoria = categorias_disponibles[categoria]
        for producto in lista_productos:
            precio = round(random.uniform(0.50, 50.00), 2)  # Precio aleatorio entre 1 y 50
            stock = random.randint(0, 500)  # Stock aleatorio entre 10 y 100 unidades
            
            cursor.execute("INSERT INTO productos (nombre_producto, precio, stock, id_categoria) VALUES (%s, %s, %s, %s)",
                           (producto, precio, stock, id_categoria))
    else:
        print(f"La categoría '{categoria}' no existe en la base de datos.")

conexion6.commit()
cursor.close()
conexion6.close()

print("Todos los productos han sido insertados correctamente con sus categorías correspondientes.")

Todos los productos han sido insertados correctamente con sus categorías correspondientes.


Clientes

In [13]:
# Llenamos la tabla clientes
conexion7 = con.connect(
    host="localhost",
    port= "3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion7.cursor()

#Nombres y apellidos clientes
nom_clientes = [f"Cliente{i}" for i in range(1, 2001)]
ape_clientes = [f"Apellido" for _ in range(2000)]

# email clientes
mail_clientes = [f"{nombre}.{apellido}@python.com" for nombre, apellido in zip(nom_clientes, ape_clientes)]

# Telefono clientes
tel_clientes = [random.randint(600000000, 699999999) for _ in range(2000)]

#Direccion clientes
dir_clientes = [f"Calle_cliente_{i}" for i in range(1, 2001)]

#Codigo postal
cod_postal = [random.randint(10000, 99999) for _ in range(2000)]


# Actualizar cada empleado con un puesto y un id_tienda aleatorio
for i in range(2000):
    cursor.execute("INSERT INTO clientes (nombre, apellido, email, telefono, direccion, codigo_postal) VALUES (%s, %s, %s, %s, %s, %s)",
                   (nom_clientes[i], ape_clientes[i], mail_clientes[i], tel_clientes[i], dir_clientes[i], cod_postal[i]))

conexion7.commit()
cursor.close()
conexion7.close()

print("Los 2000 clientes han sido insertados correctamente con todos sus datos.")

Los 2000 clientes han sido insertados correctamente con todos sus datos.


Ordenes

In [14]:
conexion8 = con.connect(
    host="localhost",
    port="3306",
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion8.cursor()

# Obtener lista de ID de clientes
cursor.execute("SELECT id_cliente FROM clientes")
id_clientes = [row[0] for row in cursor.fetchall()]

# Obtener lista de ID de empleados
cursor.execute("SELECT id_empleado FROM empleados")
id_empleados = [row[0] for row in cursor.fetchall()]

# Generar 10,000 fechas aleatorias en 2024
fechas = [datetime.datetime(year=2024, month=1, day=1) + datetime.timedelta(days=random.randint(0, 364)) for _ in range(10000)]

# Insertar los datos en la tabla ordenes
for _ in range(10000): 
    id_cliente = random.choice(id_clientes) 
    id_empleado = random.choice(id_empleados) 
    fecha_orden = random.choice(fechas)  
    metodo_pago = random.choice(["Tarjeta", "Efectivo"])

    cursor.execute(
        "INSERT INTO ordenes (id_cliente, id_empleado, fecha_orden, metodo_pago) VALUES (%s, %s, %s, %s)",
        (id_cliente, id_empleado, fecha_orden, metodo_pago)
    )

conexion8.commit()
cursor.close()
conexion8.close()

print("10,000 órdenes han sido insertadas correctamente")

✅ 10,000 órdenes han sido insertadas correctamente en la base de datos.


Detalles orden

In [ ]:
conexion9 = con.connect(
    host="localhost",
    port=3306,  # Número sin comillas
    user="root",
    password="admin",
    database="supermercado"
)
cursor = conexion9.cursor()

# Obtener lista de ID de órdenes existentes
cursor.execute("SELECT id_orden FROM ordenes")
id_ordenes = [row[0] for row in cursor.fetchall()]

# Obtener lista de productos junto con su precio
cursor.execute("SELECT id_producto, precio_unitario FROM productos")
productos_info = {row[0]: row[1] for row in cursor.fetchall()}  # Diccionario {id_producto: precio_unitario}

# Insertar los datos en la tabla ordenes
for _ in range(30000): 
    id_orden = random.choice(id_ordenes) 
    id_producto = random.choice(list(productos_info.keys()))  # Escoge un producto válido
    cantidad = random.randint(1, 50)  
    precio_unitario = productos_info[id_producto]  # Obtiene el precio del producto
    precio_total = cantidad * precio_unitario  # Calcula el precio total

    cursor.execute(
        "INSERT INTO ordenes (id_orden, id_producto, cantidad, precio_unitario, precio_total) VALUES (%s, %s, %s, %s, %s)",
        (id_orden, id_producto, cantidad, precio_unitario, precio_total)
    )

# Confirmar cambios
conexion9.commit()
cursor.close()
conexion9.close()

print("Se han insertado 30,000 registros exitosamente con precio unitario y total.")